In [170]:
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [171]:
df = pd.read_csv('train.csv', encoding='iso-8859-1')

In [172]:
df.head()


,InterviewerId,Score_A_Interviewer_1,Score_B,Desc_A,Desc_B,Prio_A,Prio_B,Years of exp,Hire_call_interviewer_1,Interviewee
0,Prashant Menon,6,5.0,He seems to be not in touch with Object Orient...,Talks less. Finds it a bit difficult to commu...,10,6,5,0,Ambuj kumar Sharma
1,Rahul Kumar,7,5.0,"He wrote the code in java, a bit slow in writi...",He said he is working on some ML algo to class...,10,6,4,1,Arshan
2,Prasanth Menon,8,7.0,He thought about different datastructures that...,Good,10,6,4,1,NaN
3,NaN,7,5.0,He asked the right questions to begin with. He...,He talked about seller recall product he worke...,10,6,4,1,Arshan
4,Shyam,4,5.0,He took some time in understanding the problem...,NaN,10,6,6,0,Mayank Rathore


In [173]:
df = df[['Desc_A','Hire_call_interviewer_1']]

In [174]:
df.head()

,Desc_A,Hire_call_interviewer_1
0,He seems to be not in touch with Object Orient...,0
1,"He wrote the code in java, a bit slow in writi...",1
2,He thought about different datastructures that...,1
3,He asked the right questions to begin with. He...,1
4,He took some time in understanding the problem...,0


In [175]:
y = df['Hire_call_interviewer_1']
xtrain, xvalid, ytrain, yvalid = train_test_split(df.Desc_A.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [176]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [177]:
xtrain_tfv

<4x10 sparse matrix of type '<class 'numpy.float64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [178]:
embedding_vecor_length = 32
max_feedback_length = 10
model = Sequential()
model.add(Embedding(100, embedding_vecor_length, input_length=max_feedback_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [179]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 10, 32)            3200      
_________________________________________________________________
lstm_16 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 101       
Total params: 56,501
Trainable params: 56,501
Non-trainable params: 0
_________________________________________________________________
None


In [181]:
model.fit(xtrain_tfv, ytrain, epochs=10)

# Final evaluation of the model
scores = model.evaluate(xvalid_tfv, yvalid, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
4/4 [==============================] - 0s 0us/step - loss: 0.6931 - acc: 0.5000
Epoch 2/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6931 - acc: 0.5000
Epoch 3/10
4/4 [==============================] - 0s 0us/step - loss: 0.6932 - acc: 0.5000
Epoch 4/10
4/4 [==============================] - 0s 0us/step - loss: 0.6932 - acc: 0.5000
Epoch 5/10
4/4 [==============================] - 0s 1ms/step - loss: 0.6931 - acc: 0.5000
Epoch 6/10
4/4 [==============================] - 0s 118us/step - loss: 0.6931 - acc: 0.5000
Epoch 7/10
4/4 [==============================] - 0s 0us/step - loss: 0.6931 - acc: 0.5000
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6931 - acc: 0.5000
Epoch 9/10
4/4 [==============================] - 0s 0us/step - loss: 0.6931 - acc: 0.5000
Epoch 10/10
4/4 [==============================] - 0s 0us/step - loss: 0.6931 - acc: 0.5000
Accuracy: 50.00%


In [182]:
from sklearn.linear_model import LogisticRegression
4/4 [==============================] - 0s 0us/step - loss: 0.6931 - acc: 0.500
from sklearn.metrics import accuracy_score
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % accuracy_score(yvalid, predictions))

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets